In [1]:
import sys
print(sys.version)

3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

In [4]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [5]:
import mlflow

# mlflow.set_tracking_uri("http://127.0.0.1:5000.sqlite:///mlflow.db")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment-test")

<Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1725297092935, experiment_id='4', last_update_time=1725297092935, lifecycle_stage='active', name='nyc-taxi-experiment-test', tags={}>

In [6]:
def read_dataframe(filename):
    
    df = pd.read_parquet(filename) 

    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df['trip_duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

    # creating trip duration column in seconds
    df.trip_duration = df.trip_duration.apply(lambda x: x.total_seconds() / 60)

    df = df[(df.trip_duration >=1) & (df.trip_duration <= 60)]
    categorical = ['PULocationID','DOLocationID']
#     numerical = ['trip_distance']

    df[categorical] = df[categorical].astype('str')
    
    return df


In [7]:
# Read in data and preprocess
train_filename = 'data/green_tripdata_2021-01.parquet'
val_filename = 'data/green_tripdata_2021-02.parquet'

df_train = read_dataframe(train_filename)
df_val = read_dataframe(val_filename)

In [8]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [9]:
target = 'trip_duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [22]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


10.499110712717727

In [23]:
# Saving LR model to model folder
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
df_val.head()

In [16]:
# starting a new mlflow run
with mlflow.start_run():
    mlflow.set_tag("developer","forrest")

    mlflow.log_param("train-data-path", 'data/green_tripdata_2021-01.parquet')
    mlflow.log_param("test-data-path", 'data/green_tripdata_2021-02.parquet')
    
    alpha = 0.01
    mlflow.log_param("alpha",alpha)

    lr = Lasso(alpha)
    lr.fit(X_train,y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric('rmse',rmse)
    #mlflow.log_artifact(local_path='models/lin_reg.bin',artifact_path='models_pickle') # this saves the model in artifact path folder
    

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/09 00:06:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run wise-panda-564 at: http://127.0.0.1:5000/#/experiments/4/runs/cd9a1d668afc4cff9b1052215c202623.
2024/09/09 00:06:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4.


### Using XGBoost to tune hyperparams and log to MLFlow

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [18]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals = [(valid,"validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val,y_pred,squared=False)
        mlflow.log_metric("rmse",rmse)
    return {'loss':rmse, 'status': STATUS_OK}

In [19]:
search_space = {
    'max_depth':scope.int(hp.quniform('max_depth',4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_rate': hp.loguniform('min_child_rate', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

## Takes 17min to run
# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials()
# )

In [ ]:
# Training model on params with lowest rmse

params = {
    'learning_rate': 0.20593751467273738,
    'max_depth': 5,
    'min_child_rate': 1.4527850859098408,
    'objective': 'reg:linear',
    'reg_alpha': 0.009922055874201745,
    'reg_lambda': 0.3515985223913228,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals = [(valid,"validation")],
            early_stopping_rounds=50
        )

2024/09/15 17:42:21 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/09/15 17:42:21 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '31d47de8fda24adcb8d96abb4d5be019', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:42:21] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:42:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_child_rate" } are not used.

  warnings.warn(sms

[0]	validation-rmse:10.64064
[1]	validation-rmse:9.49265
[2]	validation-rmse:8.66983
[3]	validation-rmse:8.08805
[4]	validation-rmse:7.68200
[5]	validation-rmse:7.39908
[6]	validation-rmse:7.19965
[7]	validation-rmse:7.06098
[8]	validation-rmse:6.96287
[9]	validation-rmse:6.89004
[10]	validation-rmse:6.83630
[11]	validation-rmse:6.79317
[12]	validation-rmse:6.76207
[13]	validation-rmse:6.73802
[14]	validation-rmse:6.71729
[15]	validation-rmse:6.70331
[16]	validation-rmse:6.68393
[17]	validation-rmse:6.67178
[18]	validation-rmse:6.66128
[19]	validation-rmse:6.65465
[20]	validation-rmse:6.64558
[21]	validation-rmse:6.63477
[22]	validation-rmse:6.62712
[23]	validation-rmse:6.62046
[24]	validation-rmse:6.61247
[25]	validation-rmse:6.60874
[26]	validation-rmse:6.59975
[27]	validation-rmse:6.59361
[28]	validation-rmse:6.58961
[29]	validation-rmse:6.58531
[30]	validation-rmse:6.58166
[31]	validation-rmse:6.57544
[32]	validation-rmse:6.57266
[33]	validation-rmse:6.56767
[34]	validation-rmse:6.

In [12]:
# if set to true above it will still pertain to xgboost models further down 
mlflow.xgboost.autolog(disable=True)

In [23]:
# another way to log model
with mlflow.start_run():
    best_params = {
    'learning_rate': 0.20593751467273738,
    'max_depth': 5,
    'min_child_rate': 1.4527850859098408,
    'objective': 'reg:linear',
    'reg_alpha': 0.009922055874201745,
    'reg_lambda': 0.3515985223913228,
    'seed': 42
    }

    mlflow.log_params(best_params)
    
    # mlflow.xgboost.autolog()
    
    booster = xgb.train(
                params=best_params,
                dtrain=train,
                num_boost_round=1000,
                evals = [(valid,"validation")],
                early_stopping_rounds=50
            )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val,y_pred,squared=False)
    mlflow.log_metric("rmse",rmse)

    with open("models/preprocessor.b","wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b",artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

[0]	validation-rmse:10.64064
[1]	validation-rmse:9.49265
[2]	validation-rmse:8.66983
[3]	validation-rmse:8.08805
[4]	validation-rmse:7.68200
[5]	validation-rmse:7.39908
[6]	validation-rmse:7.19965
[7]	validation-rmse:7.06098
[8]	validation-rmse:6.96287
[9]	validation-rmse:6.89004
[10]	validation-rmse:6.83630
[11]	validation-rmse:6.79317
[12]	validation-rmse:6.76207


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:33:44] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:33:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_child_rate" } are not used.

  warnings.warn(smsg, UserWarning)


[13]	validation-rmse:6.73802
[14]	validation-rmse:6.71729
[15]	validation-rmse:6.70331
[16]	validation-rmse:6.68393
[17]	validation-rmse:6.67178
[18]	validation-rmse:6.66128
[19]	validation-rmse:6.65465
[20]	validation-rmse:6.64558
[21]	validation-rmse:6.63477
[22]	validation-rmse:6.62712
[23]	validation-rmse:6.62046
[24]	validation-rmse:6.61247
[25]	validation-rmse:6.60874
[26]	validation-rmse:6.59975
[27]	validation-rmse:6.59361
[28]	validation-rmse:6.58961
[29]	validation-rmse:6.58531
[30]	validation-rmse:6.58166
[31]	validation-rmse:6.57544
[32]	validation-rmse:6.57266
[33]	validation-rmse:6.56767
[34]	validation-rmse:6.55974
[35]	validation-rmse:6.55471
[36]	validation-rmse:6.54924
[37]	validation-rmse:6.54585
[38]	validation-rmse:6.54101
[39]	validation-rmse:6.53873
[40]	validation-rmse:6.53332
[41]	validation-rmse:6.52840
[42]	validation-rmse:6.52458
[43]	validation-rmse:6.51927
[44]	validation-rmse:6.51615
[45]	validation-rmse:6.51318
[46]	validation-rmse:6.51090
[47]	validatio

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:33:55] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools 

In [24]:
# copied from mlflow: Make Prediction code snippet
logged_model = 'runs:/474547fab631469dba33913e4da0996f/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(pd.DataFrame(data))

In [25]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 474547fab631469dba33913e4da0996f

In [26]:
# now it is an xgboost object and can predict values
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [28]:
y_pred = xgboost_model.predict(valid)

In [29]:
y_pred[:10]

array([16.630514,  6.768431, 16.97294 , 22.199062,  9.874943, 15.434448,
       12.473573,  8.209757,  9.295039, 16.173765], dtype=float32)

### Train 4 models for model registry

In [10]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.set_experiment("nyc-taxi-experiment-test/model-registry-example")
mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():
        
        mlflow.set_tag("developer","forrest")
        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2024/09/15 17:24:01 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-test/model-registry-example' does not exist. Creating a new experiment.
2024/09/15 17:24:03 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/09/15 17:32:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml"
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcu